In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /Users/wjones/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Data Preprocessing
path = "RottenTomatoes/DataSet/train.tsv"
df = pd.read_csv(path, sep="\t") 

# remove punctuation
df['Phrase'] = df['Phrase'].str.replace(r'[^\w\s]+', '')
# remove numbers
df['Phrase'] = df['Phrase'].str.replace(r'\d+', '')
# make it all lower case
df['Phrase'] = df['Phrase'].str.lower()
# remove non-asci characters
df.Phrase.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
#df['Phrase'] = df['Phrase'].str.split()

df['Tokenized_text'] = df['Phrase'].apply(word_tokenize) 

df['Sentiment']=df['Sentiment'].astype(int) #convert the star_rating column to int
df['NNLabels'] = df['Sentiment'].div(4)

df= df[df['Sentiment']!=2]

#df['label']=np.where(df['Sentiment']>=4,1,0) #1-Positve,0-Negative
# convert to NumPy Array
train = df['Phrase'].to_numpy()


/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_41406/3302568041.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Phrase'] = df['Phrase'].str.replace(r'[^\w\s]+', '')
/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_41406/3302568041.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Phrase'] = df['Phrase'].str.replace(r'\d+', '')


In [3]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(df['Phrase'], df['Sentiment'], test_size=0.2, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (61182,) (61182,) Test:  ((15296,), (15296,))


In [4]:
X_train,X_valid,Y_train, Y_valid = train_test_split(X_train,Y_train, test_size=0.5, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_valid.shape,Y_valid.shape))

Train:  (30591,) (30591,) Test:  ((30591,), (30591,))


In [5]:
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [6]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
tf_x_train = tfidfvectorizer.fit_transform(X_train)
tf_x_valid = tfidfvectorizer.transform(X_valid)

In [7]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

clf = LinearSVC(random_state=0)
clf.fit(tf_x_train,Y_train)
y_test_pred=clf.predict(tf_x_valid)

report=classification_report(Y_valid, y_test_pred,output_dict=True)
report

{'0': {'precision': 0.4579002079002079,
  'recall': 0.30944854232525465,
  'f1-score': 0.36931460909662545,
  'support': 2847},
 '1': {'precision': 0.690320874471086,
  'recall': 0.7175845322092916,
  'f1-score': 0.7036887271420227,
  'support': 10913},
 '3': {'precision': 0.6874407261888633,
  'recall': 0.7746564885496183,
  'f1-score': 0.7284473476419496,
  'support': 13100},
 '4': {'precision': 0.5048809058961343,
  'recall': 0.346555883141249,
  'f1-score': 0.41099809281627464,
  'support': 3731},
 'accuracy': 0.6587885325749403,
 'macro avg': {'precision': 0.5851356786140729,
  'recall': 0.5370613615563534,
  'f1-score': 0.5531121941742181,
  'support': 30591},
 'weighted avg': {'precision': 0.6448399126529842,
  'recall': 0.6587885325749403,
  'f1-score': 0.6474743522541285,
  'support': 30591}}

In [25]:
sentance = ["This is the absolute best thing ever"]
sent = tfidfvectorizer.transform(sentance)
sent_prediction = clf.predict(sent)
print("The sentiment of the input is a " + sent_prediction[0].astype(str)+" out of 4")

The sentiment of the input is a 4 out of 4


1